# Read the Source Words Dataset

In [1]:
import json # Extracting data from the .json files
from collections import defaultdict # Making higher dimensional dictionaries

def source_verb(dataset_file):
    
    # parsing the json file and storing it in an object
    with open(dataset_file, 'r', encoding = 'utf-8') as f:
        source = json.load(f)
        
    # Setting all the words in the corpus in a more accessible dictionary,
    # where dictionary maps the tag of the language to the words
    source_word = defaultdict(list)
    for x in source:
        for y in x:
            words = x[y].split()
            for word in words:
                source_word[y].append(word)
        
    return source_word

## Phase 1: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb

In [2]:
def infinitive_verb_search(source_verb, target_verb, input_verb, infinitive_verb_dataset):
    
    translation = []
    
    for i in range(len(input_verb)):
        for j in infinitive_verb_dataset.get(source_verb):
            if input_verb[i] == j:
                # Get the index of the source word and use it to get it's equivalent in target word
                translation.append(infinitive_verb_dataset.get(target_verb)[infinitive_verb_dataset.get(source_verb).index(j)])

    if len(translation) == 0:
        translation.append("")
     
    return translation

## Phase 2: If the input word isn't an infinitive verb, check what kind of verb is it. (Verb tense recognition)
### Translation (Arabic to Turkish)

In [3]:
def ar_to_tr_translation(input_verb, infinitive_verb_dataset):

    # Phase 1: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
    translation = infinitive_verb_search('ar', 'tr', input_verb, infinitive_verb_dataset)
    
    # Phase 2: If the input verb isn't an infinitive verb, check what kind of verb is it. (Verb tense recognition)
    if translation[0] == "":
        
        # Future tense (فعل المستقبل)
        if input_verb[0][0] == "س":
            
            # Phase 3: Stemming the Arabic verb from the Future Tense and the Pronoun letters
            ar_verb_stem = ar_verb_stemming(input_verb, "FT")
            
            # Phase 4: Convert the Arabic Stem verb into an Infinitive verb
            ar_stem_to_infinitive = ar_verb_stem_to_infinitive(ar_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            ar_infinitive_verb_search = infinitive_verb_search('ar', 'tr', ar_stem_to_infinitive, infinitive_verb_dataset)
            
            if ar_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Turkish infinitive verb to the correct verb tense and pronoun
                translation = tr_verb_infinitive_to_tense(ar_infinitive_verb_search, ar_verb_stem[0], "FT")
            
            
        # Present Continuous tense (فعل المضارع)
        elif input_verb[0][:2] == "أُ" or input_verb[0][0] == "ت" or input_verb[0][0] == "ي":
            
            # Phase 3: Stemming the Arabic verb from the Present Continuous Tense and the Pronoun letters
            ar_verb_stem = ar_verb_stemming(input_verb, "PCT")
            
            # Phase 4: Convert the Arabic Stem verb into an Infinitive verb
            ar_stem_to_infinitive = ar_verb_stem_to_infinitive(ar_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            ar_infinitive_verb_search = infinitive_verb_search('ar', 'tr', ar_stem_to_infinitive, infinitive_verb_dataset)
            
            if ar_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Turkish infinitive verb to the correct verb tense and pronoun
                translation = tr_verb_infinitive_to_tense(ar_infinitive_verb_search, ar_verb_stem[0], "PCT")
            

        # Past tense (فعل الماضي)
        elif input_verb[0][-2:] == "تُ" or input_verb[0][-2:] == "تَ" or input_verb[0][-2:] == "تِ" or input_verb[0][-1] == "َ" or input_verb[0][-1] == "ت":

            # Phase 3: Stemming the Arabic verb from the Past Tense and the Pronoun letters
            ar_verb_stem = ar_verb_stemming(input_verb, "PT")
            
            # Phase 4: Convert the Arabic Stem verb into an Infinitive verb
            ar_stem_to_infinitive = ar_verb_stem_to_infinitive(ar_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            ar_infinitive_verb_search = infinitive_verb_search('ar', 'tr', ar_stem_to_infinitive, infinitive_verb_dataset)
            
            if ar_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Turkish infinitive verb to the correct verb tense and pronoun
                translation = tr_verb_infinitive_to_tense(ar_infinitive_verb_search, ar_verb_stem[0], "PT")

            
    return translation

# Arabic to Turkish

## Phase 3: Verb stemming. Remove the letters related to the tenses (Future, Present, or Past) and the pronoun

In [4]:
def ar_verb_stemming(input_verb, verb_tense):
    
    # Future Tense
    if verb_tense == "FT":
        verb_variation = []
        
        # Step 1: Delete the Arabic Future Tense prefix "س" (first letter)
        verb_variation.append(input_verb[0][-(len(input_verb) - 2):]) # Delete first letter

        # Step 2: Check the Arabic Present Tense prefix AND suffix of the verb and delete the letters related to
        if verb_variation[0][0] == "ا" or verb_variation[0][0] == "أ":
            pronoun = "1PS" # 1st Person Singular Pronoun "أنا"
            verb_variation[0] = verb_variation[0][-(len(input_verb) - 2):] # Delete first letter
            #verb_variation[0] = verb_variation[0]

        elif verb_variation[0][0] == "ت":
            if verb_variation[0][-1] == "ن" and verb_variation[0][-2] == "ي": # Check last 2 letters
                pronoun = "2PFS" # 2nd Person Female Singular Pronoun "أنتِ"
                verb_variation[0] = verb_variation[0][-(len(input_verb) - 2):] # Delete first letter
                verb_variation[0] = verb_variation[0][:-2] # Delete last 2 letters

            else: 
                pronoun = "2PMS_3PFS" # 2nd Person Male Singular Pronoun "أنتَ" OR 3nd Person Female Singular Pronoun "هي" 
                verb_variation[0] = verb_variation[0][-(len(input_verb) - 2):] # Delete first letter

        elif verb_variation[0][0] == "ي":  
            pronoun = "3PMS" # 3rd Person Male Singular Pronoun "هو"
            verb_variation[0] = verb_variation[0][-(len(input_verb) - 2):] # Delete first letter
            

            
    # Present Continuous Tense
    elif verb_tense == "PCT":
        verb_variation = []
        
        # Step 1: Check the Arabic Present Tense prefix AND suffix of the verb and delete the letters related to
        if input_verb[0][:2] == "أُ":
            pronoun = "1PS" # 1st Person Singular Pronoun "أنا"
            verb_variation.append(input_verb[0][-(len(input_verb) - 3):]) # Delete first letter
            #verb_variation.append("أ" + input_verb[0][-(len(input_verb) - 3):]) # Delete second letter
            #print(verb_variation[0])
            
        elif input_verb[0][0] == "ت":
            if input_verb[0][-1] == "ن" and input_verb[0][-2] == "ي": # Check last 2 letters
                pronoun = "2PFS" # 2nd Person Female Singular Pronoun "أنتِ"
                verb_variation.append(input_verb[0][-(len(input_verb) - 2):]) # Delete first letter
                verb_variation[0] = verb_variation[0][:-2] # Delete last 2 letters
            else: 
                pronoun = "2PMS_3PFS" # 2nd Person Male Singular Pronoun "أنتَ" OR 3nd Person Female Singular Pronoun "هي" 
                verb_variation.append(input_verb[0][-(len(input_verb) - 2):]) # Delete first letter

        elif input_verb[0][0] == "ي":
            pronoun = "3PMS" # 3rd Person Male Singular Pronoun "هو"
            verb_variation.append(input_verb[0][-(len(input_verb) - 2):]) # Delete first letter
            
            
    # Past Tense
    elif verb_tense == "PT":
        verb_variation = []

        # Step 1: Check the Arabic Past Tense suffix of the verb and delete the letters related to
        if input_verb[0][-2:] == "تُ":
            pronoun = "1PS" # 1st Person Singular Pronoun "أنا"
            verb_variation.append(input_verb[0][:-2]) # Delete last 2 letters

        elif input_verb[0][-2:] == "تَ":
            pronoun = "2PMS" # 2nd Person Male Singular Pronoun "أنتَ"
            verb_variation.append(input_verb[0][:-2]) # Delete last 2 letters
        
        elif input_verb[0][-2:] == "تِ":
            pronoun = "2PFS" # 2nd Person Female Singular Pronoun "أنتِ"
            verb_variation.append(input_verb[0][:-2]) # Delete last 2 letters
            
        elif input_verb[0][-1] == "ت":
            pronoun = "3PFS" # 3rd Person Female Singular Pronoun "هي"
            verb_variation.append(input_verb[0][:-1]) # Delete last letter

        elif input_verb[0][-1] == "َ": 
            pronoun = "3PMS" # 3rd Person Male Singular Pronoun "هو"
            verb_variation.append(input_verb[0][:-1]) # Delete last letter
            

    ar_verb_stem = [pronoun, verb_variation]
    
    return ar_verb_stem

## Phase 4: Convert the Arabic Stem verb into an Infinitive verb

In [5]:
def ar_verb_stem_to_infinitive(stem_verb):
    
    infinitive_verb = stem_verb
    
    for i in range(len(infinitive_verb)):
        
        # Condition 1: If word starts with "أ"
        if infinitive_verb[i][0] == "أ" and infinitive_verb[i][2] != "ّ":
            infinitive_verb[i] = "إ" + infinitive_verb[i][1:] # Replace "أ" with "إ" 

            # Special condition
            if infinitive_verb[i][-2] != "ا": # If there is no "ا" before last letter
                infinitive_verb[i] = infinitive_verb[i][:-1] + "ا" + infinitive_verb[i][-1] # Insert "ا" before last letter
                
            elif infinitive_verb[i][-2] == "ا": # If there is "ا" before last letter
                infinitive_verb[i] = infinitive_verb[i] + "ة" # Append "ة" as the last letter
                
            elif infinitive_verb[i][-1] == "ى": # If "ى" is the last letter
                infinitive_verb[i] = infinitive_verb[i][:-1] # Delete "ى" as the last letter
                infinitive_verb[i] = infinitive_verb[i] + "ا" # Append "ا" as the last letter
                infinitive_verb[i] = infinitive_verb[i] + "ء" # Append "ء" as the last letter


        # Condition 2: If second letter has "شدة"
        elif infinitive_verb[i][1] == "ّ" or infinitive_verb[i][2] == "ّ":
            infinitive_verb[i] = "ت" + infinitive_verb[i] # Insert "ت" at the beginning
            
            # Special condition
            # If last letter is "ا" or "ى" it turns into "ي"
            # And append "ة" as the last letter
            if infinitive_verb[i][-1] == "ا" or infinitive_verb[i][-1] == "ى":
                infinitive_verb[i] = infinitive_verb[i][:-1] + "ي"
                infinitive_verb[i] = infinitive_verb[i] + "ة"
            else:
                infinitive_verb[i] = infinitive_verb[i][:-1] + "ي" + infinitive_verb[i][-1] # Insert "ي" before last letter


        # Condition 3: If second letter is "ا"
        elif infinitive_verb[i][1] == "ا":
            infinitive_verb[i] = "م" + infinitive_verb[i] # Insert "م" as the first letter
            infinitive_verb[i] = infinitive_verb[i] + "ة" # Insert "ة" as the last letter


        # Condition 4: If first letter is "ت"
        elif infinitive_verb[i][0] == "ت":
            if infinitive_verb[i][-1] == "ى": # If the last letter is "ى", replace it with "ي"
                infinitive_verb[i] = infinitive_verb[i][:-1] + "ي"
            else:
                infinitive_verb[i] = infinitive_verb[i] # Keep the verb as it is


        # Condition 5: If the length of the verb is bigger than 4 letters
        elif len(infinitive_verb[i]) > 4:
            infinitive_verb[i] = "ا" + infinitive_verb[i] # Insert "ا" as the first letter
            infinitive_verb[i] = infinitive_verb[i][:-1] + "ا" + infinitive_verb[i][-1] # Insert "ا" before last letter


        # Condition 6: If non of the above conditions is correct
        else:
            infinitive_verb[i] = infinitive_verb[i] + "ة" # Insert "ة" as the last letter
    
    
    return infinitive_verb

## Phase 6: Convert the Turkish infinitive verb to the correct verb tense and pronoun

In [6]:
def tr_verb_infinitive_to_tense(infinitive_verb, pronoun, verb_tense):
    
    for i in range(len(infinitive_verb)):

        # Step 1: Delete the Infinitive suffix "mak/mek" from the verb
        if infinitive_verb[i][-3:] == "mak":
            infinitive_verb[i] = infinitive_verb[i].removesuffix("mak")

        elif infinitive_verb[i][-3:] == "mek":
            infinitive_verb[i] = infinitive_verb[i].removesuffix("mek")


        # Step 2: Check the correct verb tense of the source verb

        # Future Tense
        if verb_tense == "FT":

            # Special Case:
            # Step 1: Check if the last letter of the target stem verb is a vowel (a, ı, o, u, e, i, ö, or ü), add the letter "y" as the last letter
            if infinitive_verb[i][-1] == "a" or infinitive_verb[i][-1] == "ı" or infinitive_verb[i][-1] == "o" or infinitive_verb[i][-1] == "u" or infinitive_verb[i][-1] == "e" or infinitive_verb[i][-1] == "i" or infinitive_verb[i][-1] == "ö" or infinitive_verb[i][-1] == "ü":
                infinitive_verb[i] = infinitive_verb[i] + "y"


            # Step 2: Add the Future Tense suffix "acağ" to the end of the target verb with the vowels (a, ı, o, u)
            if infinitive_verb[i][-2] == "a" or infinitive_verb[i][-2] == "ı" or infinitive_verb[i][-2] == "o" or infinitive_verb[i][-2] == "u":

                # Step 3: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
                if pronoun == "1PS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "acağ"

                    # Step 3: Add the pronoun suffix
                    infinitive_verb[i] = infinitive_verb[i] + "ım"

                # Condition 2: 2nd Person Female Singular Pronoun "أنتِ"|Sen"
                elif pronoun == "2PFS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "acak"

                    # Step 3: Add the pronoun suffix
                    infinitive_verb[i] = infinitive_verb[i] + "sın"

                # Condition 3: 3rd Person Male Singular Pronoun "هو"|O"
                elif pronoun == "3PMS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "acak"

                # Condition 4: 2nd Person Male Singular Pronoun "أنتَ"|Sen OR 3rd Person Female Singular Pronoun "هي"|O
                elif pronoun == "2PMS_3PFS":
                    # Step 2: Add the Future Tense suffix AND Step 3: Add the pronoun suffix
                    infinitive_verb_2PMS = infinitive_verb[i] + "acak" + "sın"
                    infinitive_verb_3PFS = infinitive_verb[i] + "acak"
                    infinitive_verb[i] = infinitive_verb_2PMS + " | " + infinitive_verb_3PFS

            # Step 2: Add the Future Tense suffix "eceğ" to the end of the target verb with the vowels (e, i, ö, ü)
            elif infinitive_verb[i][-2] == "e" or infinitive_verb[i][-2] == "i" or infinitive_verb[i][-2] == "ö" or infinitive_verb[i][-2] == "ü":

                # Step 3: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
                if pronoun == "1PS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "eceğ"

                    # Step 3: Add the pronoun suffix
                    infinitive_verb[i] = infinitive_verb[i] + "im"

                # Condition 2: 2nd Person Female Singular Pronoun "Sen"|"أنتِ"
                elif pronoun == "2PFS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "ecek"

                    # Step 3: Add the pronoun suffix
                    infinitive_verb[i] = infinitive_verb[i] + "sin"

                # Condition 3: 3rd Person Male Singular Pronoun "O"|"هو"
                elif pronoun == "3PMS":
                    # Step 2: Add the Future Tense suffix
                    infinitive_verb[i] = infinitive_verb[i] + "ecek"

                # Condition 4: 2nd Person Male Singular Pronoun "Sen"|"أنتَ" OR 3rd Person Female Singular Pronoun "O"|"هي"
                elif pronoun == "2PMS_3PFS":
                    # Step 2: Add the Future Tense suffix AND Step 3: Add the pronoun suffix
                    infinitive_verb_2PMS = infinitive_verb[i] + "ecek" + "sin"
                    infinitive_verb_3PFS = infinitive_verb[i] + "ecek"
                    infinitive_verb[i] = infinitive_verb_2PMS + " | " + infinitive_verb_3PFS



        # Present Continuous Tense
        if verb_tense == "PCT":

            # Special Case:
            # Step 1: Check if the last letter of the target stem verb is a vowel (a, ı, o, u, e, i, ö, or ü). Change the suffix based on the last letter
            if infinitive_verb[i][-1] == "a" or infinitive_verb[i][-1] == "ı" or infinitive_verb[i][-1] == "o" or infinitive_verb[i][-1] == "u" or infinitive_verb[i][-1] == "e" or infinitive_verb[i][-1] == "i" or infinitive_verb[i][-1] == "ö" or infinitive_verb[i][-1] == "ü":

                # If last letter "a" replcae it with "ı"
                # If last letter "ı" keep it as it is
                if infinitive_verb[i][-1] == "a":
                    infinitive_verb[i] = infinitive_verb[i].removesuffix("a")
                    infinitive_verb[i] = infinitive_verb[i] + "ı"

                # If last letter "o" replcae it with "u"
                # If last letter "u" keep it as it is
                elif infinitive_verb[i][-1] == "o":
                    infinitive_verb[i] = infinitive_verb[i].removesuffix("o")
                    infinitive_verb[i] = infinitive_verb[i] + "u"

                # If last letter "e" replcae it with "i"
                # If last letter "i" keep it as it is
                elif infinitive_verb[i][-1] == "e":
                    infinitive_verb[i] = infinitive_verb[i].removesuffix("e")
                    infinitive_verb[i] = infinitive_verb[i] + "i"

                # If last letter "ö" replcae it with "ü"
                # If last letter "ü" keep it as it is
                elif infinitive_verb[i][-1] == "ö":
                    infinitive_verb[i] = infinitive_verb[i].removesuffix("ö")
                    infinitive_verb[i] = infinitive_verb[i] + "ü"


                # Step 2: Add the Present Continuous Tense suffix "yor" to the end of the target verb
                infinitive_verb[i] = infinitive_verb[i] + "yor"

                # Step 3: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
                if pronoun == "1PS":
                    infinitive_verb[i] = infinitive_verb[i] + "um"

                # Condition 2: 2nd Person Female Singular Pronoun "أنتِ"|Sen"
                elif pronoun == "2PFS":
                    infinitive_verb[i] = infinitive_verb[i] + "sun"

                # Condition 3: 3rd Person Male Singular Pronoun "هو"|O"
                elif pronoun == "3PMS":
                    infinitive_verb[i] = infinitive_verb[i]

                # Condition 4: 2nd Person Male Singular Pronoun "أنتَ"|Sen OR 3rd Person Female Singular Pronoun "هي"|O
                elif pronoun == "2PMS_3PFS":
                    # Step 2: Add the Present Tense suffix AND Step 3: Add the pronoun suffix
                    infinitive_verb_2PMS = infinitive_verb[i] + "sun"
                    infinitive_verb_3PFS = infinitive_verb[i]
                    infinitive_verb[i] = infinitive_verb_2PMS + " | " + infinitive_verb_3PFS



            # Step 2: Add the Present Continuous Tense suffix "Iyor" to the end of the target verb with the vowels (a, ı, o, u, e, i, ö, or ü)
            elif infinitive_verb[i][-2] == "a" or infinitive_verb[i][-2] == "ı" or infinitive_verb[i][-2] == "o" or infinitive_verb[i][-2] == "u" or infinitive_verb[i][-1] == "e" or infinitive_verb[i][-1] == "i" or infinitive_verb[i][-1] == "ö" or infinitive_verb[i][-1] == "ü":

                # Step 3: Add one of the vowel suffixes (ı, u, i, or ü) based on the vowel before the last letter
                if infinitive_verb[i][-2] == "a" or infinitive_verb[i][-2] == "ı":
                    infinitive_verb[i] = infinitive_verb[i] + "ı"

                elif infinitive_verb[i][-2] == "o" or infinitive_verb[i][-2] == "u":
                    infinitive_verb[i] = infinitive_verb[i] + "u"

                elif infinitive_verb[i][-2] == "e" or infinitive_verb[i][-2] == "i":
                    infinitive_verb[i] = infinitive_verb[i] + "i"

                elif infinitive_verb[i][-2] == "ö" or infinitive_verb[i][-2] == "ü":
                    infinitive_verb[i] = infinitive_verb[i] + "ü"

                # Step 4: Add Present Continuous Tense suffix "yor"
                infinitive_verb[i] = infinitive_verb[i] + "yor"

                # Step 5: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
                if pronoun == "1PS":
                    infinitive_verb[i] = infinitive_verb[i] + "um"

                # Condition 2: 2nd Person Female Singular Pronoun "أنتِ"|Sen"
                elif pronoun == "2PFS":
                    infinitive_verb[i] = infinitive_verb[i] + "sun"

                # Condition 3: 3rd Person Male Singular Pronoun "هو"|O"
                elif pronoun == "3PMS":
                    infinitive_verb[i] = infinitive_verb[i]

                # Condition 4: 2nd Person Male Singular Pronoun "أنتَ"|Sen OR 3rd Person Female Singular Pronoun "هي"|O
                elif pronoun == "2PMS_3PFS":
                    infinitive_verb_2PMS = infinitive_verb[i] + "sun"
                    infinitive_verb_3PFS = infinitive_verb[i]
                    infinitive_verb[i] = infinitive_verb_2PMS + " | " + infinitive_verb_3PFS



        # Past Tense
        if verb_tense == "PT":

            # Special Case:
            # Step 1: Check if the last letter of the target stem verb is (ç, f, h, k, s, ş, t, or p). Change the suffix based on the last letter
            if infinitive_verb[i][-1] == "ç" or infinitive_verb[i][-1] == "f" or infinitive_verb[i][-1] == "h" or infinitive_verb[i][-1] == "k" or infinitive_verb[i][-1] == "s" or infinitive_verb[i][-1] == "ş" or infinitive_verb[i][-1] == "t" or infinitive_verb[i][-1] == "p":
                # Step 1: Add Past Tense suffix
                infinitive_verb[i] = infinitive_verb[i] + "t"

            else:
                # Step 1: Add Past Tense suffix
                infinitive_verb[i] = infinitive_verb[i] + "d"


            # Step 2: Add one of the vowel suffixes (ı, u, i, or ü) based on the last vowel in the verb
            if infinitive_verb[i][-2] == "a" or infinitive_verb[i][-3] == "a" or infinitive_verb[i][-2] == "ı" or infinitive_verb[i][-3] == "ı":
                infinitive_verb[i] = infinitive_verb[i] + "ı"

            elif infinitive_verb[i][-2] == "o" or infinitive_verb[i][-3] == "o" or infinitive_verb[i][-2] == "u" or infinitive_verb[i][-3] == "u":
                infinitive_verb[i] = infinitive_verb[i] + "u"

            elif infinitive_verb[i][-2] == "e" or infinitive_verb[i][-3] == "e" or infinitive_verb[i][-2] == "i" or infinitive_verb[i][-3] == "i":
                infinitive_verb[i] = infinitive_verb[i] + "i"

            elif infinitive_verb[i][-2] == "ö" or infinitive_verb[i][-3] == "ö" or infinitive_verb[i][-2] == "ü" or infinitive_verb[i][-3] == "ü":
                infinitive_verb[i] = infinitive_verb[i] + "ü"


            # Step 3: Check the correct pronoun of the source verb
            # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
            if pronoun == "1PS":
                infinitive_verb[i] = infinitive_verb[i] + "m"

            # Condition 2: 2nd Person Female Singular Pronoun "أنتِ"|Sen" OR 2nd Person Male Singular Pronoun "أنتَ"|Sen
            elif pronoun == "2PFS" or pronoun == "2PMS":
                infinitive_verb[i] = infinitive_verb[i] + "n"

            # Condition 3: 3rd Person Male Singular Pronoun "هو"|O" OR 3rd Person Female Singular Pronoun "هي"|O
            elif pronoun == "3PMS" or pronoun == "3PFS":
                infinitive_verb[i] = infinitive_verb[i]

                
    return infinitive_verb

In [7]:
def main():

    dataset_file = 'data.json' #data file with the translations
    
    source_verb_dataset = source_verb(dataset_file) # Read the dataset and tag each word with its language
    
    # Translation Program
    print("Welcome to the Arabic/Turkish Translator!")
    print("Would you like to translate from: \n 1) Arabic to Turkish \n 2) Turkish to Arabic")
    translate_choice = input()
    
    if translate_choice == "1":
        print("Enter the verb:")
        print("AR:")
        input_verb = [input()] # Placing the verb written by the user in a list
        verb_search = ar_to_tr_translation(input_verb, source_verb_dataset) # Predict the verb!
        print("TR:")
        print(verb_search)
        
    elif translate_choice == "2":
        print("Enter the verb:")
        print("TR:")
        input_verb = [input()] # Placing the verb written by the user in a list
        verb_search = tr_to_ar_translation(input_verb, source_verb_dataset) # Predict the verb!
        print("AR:")
        print(verb_search)

main()

Welcome to the Arabic/Turkish Translator!
Would you like to translate from: 
 1) Arabic to Turkish 
 2) Turkish to Arabic
1
Enter the verb:
AR:
شاهدت
TR:
['izledi']
